<a href="https://colab.research.google.com/github/AkshatBhatnagar29/Topsis_Implementation_102303158/blob/main/Topsis_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%writefile topsis.py
import sys
import pandas as pd
import numpy as np
import os

def main():
    if len(sys.argv) != 5:
        print("Error: Wrong number of parameters")
        print("Usage: python topsis.py <InputDataFile> <Weights> <Impacts> <OutputResultFileName>")
        return

    input_file = sys.argv[1]
    weights_str = sys.argv[2]
    impacts_str = sys.argv[3]
    output_file = sys.argv[4]

    if not os.path.isfile(input_file):
        print("Error: File not found")
        return

    try:
        df = pd.read_csv(input_file)
    except Exception:
        print("Error: Could not read file")
        return

    if df.shape[1] < 3:
        print("Error: Input file must contain three or more columns")
        return

    try:
        data = df.iloc[:, 1:].values.astype(float)
    except ValueError:
        print("Error: From 2nd to last columns must contain numeric values only")
        return

    try:
        weights = [float(w) for w in weights_str.split(',')]
        impacts = impacts_str.split(',')
    except ValueError:
        print("Error: Weights must be numeric and separated by comma")
        return

    if len(weights) != data.shape[1] or len(impacts) != data.shape[1]:
        print("Error: Number of weights, impacts and columns must be same")
        return

    if not all(i in ['+', '-'] for i in impacts):
        print("Error: Impacts must be either + or -")
        return

    normalized_data = data / np.sqrt((data**2).sum(axis=0))

    weighted_data = normalized_data * weights

    ideal_best = []
    ideal_worst = []

    for i in range(len(impacts)):
        if impacts[i] == '+':
            ideal_best.append(max(weighted_data[:, i]))
            ideal_worst.append(min(weighted_data[:, i]))
        else:
            ideal_best.append(min(weighted_data[:, i]))
            ideal_worst.append(max(weighted_data[:, i]))

    s_best = np.sqrt(((weighted_data - ideal_best) ** 2).sum(axis=1))
    s_worst = np.sqrt(((weighted_data - ideal_worst) ** 2).sum(axis=1))

    scores = s_worst / (s_best + s_worst)

    df['Topsis Score'] = scores
    df['Rank'] = df['Topsis Score'].rank(ascending=False)

    df.to_csv(output_file, index=False)
    print(f"Result saved to {output_file}")

if __name__ == "__main__":
    main()

Writing topsis.py


In [3]:
%%writefile data.csv
Model,Storage,Camera,Price,Looks
M1,16,12,250,5
M2,32,16,300,4
M3,16,8,200,3
M4,32,20,400,5
M5,16,16,280,3

Writing data.csv


In [4]:
!python topsis.py data.csv "1,1,1,2" "+,+,-,+" result.csv

Result saved to result.csv
